# Introduction

a fine tuning notebook example using huggingface with torch on MPS Apple silicon

Reference:
* https://huggingface.co/docs/transformers/training

* Torch FT example https://colab.research.google.com/github/huggingface/notebooks/blob/main/transformers_doc/en/pytorch/training.ipynb
* TensorFlow FT example https://colab.research.google.com/github/huggingface/notebooks/blob/main/transformers_doc/en/tensorflow/training.ipynb


Example of empty GPU memory
```python
del model
del trainer
torch.cuda.empty_cache()
```

In [1]:
from util.accelerator_utils import AcceleratorHelper, DIR_MODE_MAP
import os

# set up the torch mps environment and huggingface cache home, before importing datasets and transformers
AcceleratorHelper.init_mps_torch(dir_setting=DIR_MODE_MAP.get("mac_local"))
os.environ['XDG_CACHE_HOME']

'/Users/yingding/MODELS'

In [8]:
from datasets import load_dataset # huggingface datasets

# download the dataset to $HOME/.cache/huggingface/datasets
dataset = load_dataset("yelp_review_full")
sample_id = 100
data_sample_1 = dataset["train"][sample_id]

data_sample_1


{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [14]:
import re

def count_tokens(sentence: str) -> int:
    # split the text into words, split on \W+ (non-word characters)
    words = re.split(r'\W+', sentence.strip())
    return len(words)

print(count_tokens(data_sample_1.get("text", "").strip()))

240


In [23]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [24]:
def tokenize_function(examples: dict):
    # tokenizer.model_max_length is the max length of the sequence that the model can accept
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [26]:
tokenized_data_sample_1 = tokenized_dataset.get("train")[sample_id]
tokens_data_sample_1 = tokenized_data_sample_1.get("input_ids")
# we added padding to original tokens so the length is 512
print(len(tokens_data_sample_1))

512


In [28]:
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

In [29]:
from transformers import AutoModelForSequenceClassification
# Start by loading your model and specify the number of expected labels. 
# From the Yelp Review dataset card, you know there are five labels:
# https://huggingface.co/datasets/yelp_review_full#data-fields
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [32]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [33]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [34]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [35]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [37]:
# fine-tune the model
result = trainer.train()

  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.1290173530578613, 'eval_accuracy': 0.534, 'eval_runtime': 19.6536, 'eval_samples_per_second': 50.881, 'eval_steps_per_second': 6.36, 'epoch': 1.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.2491713762283325, 'eval_accuracy': 0.546, 'eval_runtime': 19.646, 'eval_samples_per_second': 50.901, 'eval_steps_per_second': 6.363, 'epoch': 2.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.398514986038208, 'eval_accuracy': 0.568, 'eval_runtime': 19.9737, 'eval_samples_per_second': 50.066, 'eval_steps_per_second': 6.258, 'epoch': 3.0}
{'train_runtime': 298.0765, 'train_samples_per_second': 10.065, 'train_steps_per_second': 1.258, 'train_loss': 0.5611138509114584, 'epoch': 3.0}


In [38]:
result

TrainOutput(global_step=375, training_loss=0.5611138509114584, metrics={'train_runtime': 298.0765, 'train_samples_per_second': 10.065, 'train_steps_per_second': 1.258, 'train_loss': 0.5611138509114584, 'epoch': 3.0})

In [39]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,